In [1]:
import findspark
findspark.init()

import pyspark
findspark.find()

'c:\\spark'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import collections

conf = pyspark.SparkConf().setAppName('MinTemperatures').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

In [4]:
lines = sc.textFile("1800.csv")

parsedLines = lines.map(parseLine)

minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))

results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F


In [5]:
maxTemps = parsedLines.filter(lambda x: "TMAX" in x[1])
stmaxTemps = maxTemps.map(lambda x: (x[0], x[2]))
maxTemps = stmaxTemps.reduceByKey(lambda x, y: max(x,y))

maxvalues = maxTemps.collect();

for mv in maxvalues:
    print(mv[0] + "\t{:.2f}F".format(mv[1]))

ITE00100554	90.14F
EZE00100082	90.14F
